In [2]:
# pip install torch

In [4]:
# pip install transformers

In [2]:
# pip install datasets

In [4]:
# pip install torch==2.1.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [2]:
# pip install transformers==4.36.2

In [5]:
import pandas as pd
import torch
from datasets import Dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-04-07 13:39:52.676158: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
# 1. Загрузка данных
df = pd.read_csv('data/train.csv')

# 2. Разделение на train и valid
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [7]:
# 3. Преобразование в Huggingface Datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

In [8]:
# 4. Токенизатор
model_checkpoint = "distilbert/distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding='max_length', max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
valid_dataset = valid_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 1182/1182 [00:00<00:00, 9362.73 examples/s]


In [10]:
# 5. Удалим старые колонки и приведём к нужному формату
train_dataset = train_dataset.remove_columns(["text", "__index_level_0__"])
valid_dataset = valid_dataset.remove_columns(["text", "__index_level_0__"])
train_dataset.set_format("torch")
valid_dataset.set_format("torch")

In [11]:
# 6. Загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# pip install transformers[torch]

In [12]:
# pip install -U accelerate==0.25.0

In [13]:
# 7. Тренировочные параметры
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda p: {
        "accuracy": accuracy_score(p.label_ids, p.predictions.argmax(axis=1)),
        "f1": f1_score(p.label_ids, p.predictions.argmax(axis=1)),
    }
)
# 9. Обучение
trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
 97%|█████████▋| 36/37 [01:02<00:01,  1.78s/it]
                                                 A
100%|██████████| 37/37 [01:04<00:00,  1.74s/it]
                                               

{'eval_loss': 0.20527902245521545, 'eval_accuracy': 0.8934010152284264, 'eval_f1': 0.8908145580589255, 'eval_runtime': 65.7955, 'eval_samples_per_second': 17.965, 'eval_steps_per_second': 0.562, 'epoch': 1.0}


IOStream.flush timed out
 56%|█████▋    | 500/888 [28:45<19:55,  3.08s/it]  

{'loss': 0.2725, 'learning_rate': 8.738738738738739e-06, 'epoch': 1.69}


 97%|█████████▋| 36/37 [01:02<00:01,  1.80s/it]
                                                 A
100%|██████████| 37/37 [01:04<00:00,  1.76s/it]
                                               

...

 97%|█████████▋| 36/37 [01:02<00:01,  1.79s/it]
                                                 A
100%|██████████| 37/37 [01:04<00:00,  1.75s/it]
                                               

{'eval_loss': 0.17834195494651794, 'eval_accuracy': 0.9094754653130288, 'eval_f1': 0.9119341563786008, 'eval_runtime': 66.1342, 'eval_samples_per_second': 17.873, 'eval_steps_per_second': 0.559, 'epoch': 3.0}


100%|██████████| 888/888 [53:10<00:00,  3.59s/it]

{'train_runtime': 3190.0962, 'train_samples_per_second': 4.445, 'train_steps_per_second': 0.278, 'train_loss': 0.23353415136938696, 'epoch': 3.0}


TrainOutput(global_step=888, training_loss=0.23353415136938696, metrics={'train_runtime': 3190.0962, 'train_samples_per_second': 4.445, 'train_steps_per_second': 0.278, 'train_loss': 0.23353415136938696, 'epoch': 3.0})

In [15]:
# 10. Предсказания и метрики
preds = trainer.predict(valid_dataset)
y_pred = preds.predictions.argmax(axis=1)
y_true = preds.label_ids

100%|██████████| 37/37 [01:04<00:00,  1.73s/it]


In [16]:
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print("Accuracy:", acc)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Accuracy: 0.9094754653130288
F1 Score: 0.9119341563786008
Confusion Matrix:
 [[521  65]
 [ 42 554]]


In [17]:
trainer.save_model("./results/best_model")
tokenizer.save_pretrained("./results/best_model")

('./results/best_model/tokenizer_config.json',
 './results/best_model/special_tokens_map.json',
 './results/best_model/vocab.json',
 './results/best_model/merges.txt',
 './results/best_model/added_tokens.json',
 './results/best_model/tokenizer.json')

### 2. MLM 

In [2]:
# pip install transformers datasets accelerate

In [1]:
import nltk
import math
import torch
import os
from transformers import (
    RobertaTokenizerFast,
    RobertaForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset, Dataset

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2025-04-07 17:38:22.073663: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_checkpoint  = "results/best_model"
model  = RobertaForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at results/best_model and are newly initialized: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

### создаем датасет

In [5]:
# nltk.download("punkt")

In [4]:
with open("data/train-test.txt"  , "r", encoding="utf-8") as f:
    text = f.read()

In [5]:
sentences = nltk.sent_tokenize(text)
dataset = Dataset.from_dict({"text": sentences})

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False)

tokenized = dataset.map(tokenize_function, batched=True, num_proc=8, remove_columns=["text"])

Map (num_proc=8): 100%|██████████| 29347/29347 [00:01<00:00, 19681.74 examples/s]


In [7]:
block_size=128
def group_texts(examples):
    concatenated = sum(examples["input_ids"], [])
    total_length = (len(concatenated) // block_size) * block_size
    result = {
        "input_ids": [concatenated[i : i + block_size] for i in range(0, total_length, block_size)],
    }
    result["attention_mask"] = [[1] * block_size] * len(result["input_ids"])
    return result

lm_datasets = tokenized.map(group_texts, batched=True, num_proc=8)

Map (num_proc=8): 100%|██████████| 29347/29347 [00:01<00:00, 16931.81 examples/s]


In [8]:
split = lm_datasets.train_test_split(test_size=0.1)
train_dataset = split["train"]
eval_dataset = split["test"]

In [9]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [10]:
training_args = TrainingArguments(
    output_dir="mlm-results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### метрики 

In [11]:
eval_loss_before = trainer.evaluate(eval_dataset=eval_dataset).get("eval_loss")
perplexity_before = math.exp(eval_loss_before) if eval_loss_before else None
print(f"Before training: Loss = {eval_loss_before:.4f}, Perplexity = {perplexity_before:.2f}")


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 134/134 [02:16<00:00,  1.02s/it]

Before training: Loss = 17.5383, Perplexity = 41378615.23


In [12]:
trainer.train()

  3%|▎         | 100/3600 [05:11<3:01:05,  3.10s/it]

{'loss': 2.583, 'learning_rate': 4.8611111111111115e-05, 'epoch': 0.08}


  6%|▌         | 200/3600 [10:23<2:55:39,  3.10s/it]

{'loss': 1.5792, 'learning_rate': 4.722222222222222e-05, 'epoch': 0.17}


  8%|▊         | 300/3600 [15:34<2:52:36,  3.14s/it]

...

100%|██████████| 3600/3600 [3:18:24<00:00,  2.48s/it]

{'loss': 0.8326, 'learning_rate': 0.0, 'epoch': 3.0}



 99%|█████████▉| 133/134 [02:04<00:00,  1.06it/s]
                                                     
100%|██████████| 3600/3600 [3:20:29<00:00,  3.34s/it]

{'eval_loss': 0.7373327612876892, 'eval_runtime': 125.2696, 'eval_samples_per_second': 17.027, 'eval_steps_per_second': 1.07, 'epoch': 3.0}
{'train_runtime': 12029.9461, 'train_samples_per_second': 4.785, 'train_steps_per_second': 0.299, 'train_loss': 1.0613170199924045, 'epoch': 3.0}


TrainOutput(global_step=3600, training_loss=1.0613170199924045, metrics={'train_runtime': 12029.9461, 'train_samples_per_second': 4.785, 'train_steps_per_second': 0.299, 'train_loss': 1.0613170199924045, 'epoch': 3.0})

In [13]:
eval_loss_after = trainer.evaluate(eval_dataset=eval_dataset).get("eval_loss")
perplexity_after = math.exp(eval_loss_after) if eval_loss_after else None
print(f"After training:  Loss = {eval_loss_after:.4f}, Perplexity = {perplexity_after:.2f}")

100%|██████████| 134/134 [02:04<00:00,  1.08it/s]

After training:  Loss = 0.7448, Perplexity = 2.11


In [15]:
# сохраняем модель 
model.save_pretrained("mlm_results")
tokenizer.save_pretrained("mlm_results")

('mlm_results/tokenizer_config.json',
 'mlm_results/special_tokens_map.json',
 'mlm_results/vocab.json',
 'mlm_results/merges.txt',
 'mlm_results/added_tokens.json',
 'mlm_results/tokenizer.json')

### Перетренировка классификатора с использованием весов из MLM

#### классификационная голова -- results/best_model
#### веса языковой части -- mlm_results

In [3]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np
import torch

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2025-04-08 14:13:16.765579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
dataset = load_dataset("csv", data_files="data/train.csv")["train"]
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("mlm_results")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [16]:
# модель классификации (голова)
model = AutoModelForSequenceClassification.from_pretrained("results/best_model")

# MLM веса 
mlm_model = AutoModel.from_pretrained("mlm_results")

# подставляем веса в классификационную модель
model.roberta = mlm_model


Some weights of RobertaModel were not initialized from the model checkpoint at mlm_results and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {
        "accuracy": acc,
        "f1": f1,
    }

In [18]:
training_args = TrainingArguments(
    output_dir="./final_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

 99%|█████████▊| 73/74 [00:13<00:00,  5.52it/s]
                                                  
100%|██████████| 74/74 [00:13<00:00,  5.67it/s]
                                               

{'eval_loss': 0.16351749002933502, 'eval_accuracy': 0.9179357021996616, 'eval_f1': 0.9214574898785427, 'eval_runtime': 13.702, 'eval_samples_per_second': 86.265, 'eval_steps_per_second': 5.401, 'epoch': 1.0}


IOStream.flush timed out
 42%|████▏     | 500/1184 [08:19<10:08,  1.12it/s]  

{'loss': 0.1841, 'learning_rate': 1.1554054054054056e-05, 'epoch': 1.69}


 99%|█████████▊| 73/74 [00:13<00:00,  5.21it/s]
                                                  
100%|██████████| 74/74 [00:13<00:00,  5.40it/s]
                                               

...

IOStream.flush timed out
 99%|█████████▊| 73/74 [00:13<00:00,  5.60it/s]
                                                  
100%|██████████| 74/74 [00:13<00:00,  5.75it/s]
                                               

{'eval_loss': 0.17952771484851837, 'eval_accuracy': 0.9382402707275804, 'eval_f1': 0.9396195202646817, 'eval_runtime': 13.7413, 'eval_samples_per_second': 86.018, 'eval_steps_per_second': 5.385, 'epoch': 3.0}


IOStream.flush timed out
 84%|████████▍ | 1000/1184 [17:54<02:30,  1.22it/s] 

{'loss': 0.1308, 'learning_rate': 3.1081081081081082e-06, 'epoch': 3.38}


 95%|█████████▌| 1125/1184 [19:37<00:50,  1.18it/s]

In [7]:
metrics = trainer.evaluate()
metrics

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 74/74 [00:30<00:00,  2.40it/s]


{'eval_loss': 0.17952771484851837,
 'eval_accuracy': 0.9382402707275804,
 'eval_f1': 0.9382151934632695,
 'eval_runtime': 31.8946,
 'eval_samples_per_second': 37.06,
 'eval_steps_per_second': 2.32}

In [8]:
predictions = trainer.predict(tokenized_datasets["test"])
y_pred = np.argmax(predictions.predictions, axis=-1)
y_true = predictions.label_ids
print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred))

100%|██████████| 74/74 [00:30<00:00,  2.42it/s]

Confusion matrix:
[[541  47]
 [ 26 568]]


In [9]:
trainer.save_model("./final_results/best_model")